# Initializing your environment

### Setting up a virtual environment
(Optional but recommended)

```bash
virtualenv -p python3 venv
source venv/bin/activate
```
(Use `deactivate` to exit from `source` once you are done)

Alternatively you can also prefix all your `python` and `pip` commands with `./venv/bin/` (e.g: `./venv/bin/pip3 install -U pip`)

## Setting up Jupyter

In order to follow along on your computer:

```bash
pip3 install notebook
jupyter-notebook
```

## Installation of PyMISPGalaxies

#### Make sure the submodules are up-to-date and cloned

```bash
git submodule update --init --recursive PyMISPGalaxies/
```

#### Installation

```bash
cd PyMISPGalaxies
pip3 install .
```

# Basics

## Galaxies information

In [ ]:
from pymispgalaxies import Galaxies

galaxies = Galaxies()
print(list(galaxies.keys()))

## Get information on a specific Galaxy

In [ ]:
# Pick whichever you want
galaxy_type = 'Stealer'

### Standard information on the Galaxy

In [ ]:
galaxy = galaxies.get(galaxy_type)

for feature in ('type', 'name', 'description', 'version'):
    print(f'{feature}: {getattr(galaxy, feature)}')

import json
print(json.dumps(galaxy.galaxy, indent=4))

## Clusters information

In [ ]:
from pymispgalaxies import Clusters

clusters = Clusters()
print(list(clusters.keys()))

### Get the tag names associated with a cluster

In [ ]:
print(clusters.get(galaxy_type.lower()).machinetags)

### Search for a specific name

In [ ]:
for cluster, values in clusters.search('apt33'):
    print(f'{cluster.name}:')
    for value in values:
        print(f' - {value.value}: {value.description}')
    print()

## Get information on a specific Cluster

In [ ]:
# Pick whichever you want
cluster_name = 'DarkEye'

### Standard information on a Cluster value

In [ ]:
cluster_value = clusters.get(galaxy_type.lower()).get(cluster_name)

for feature in ('value', 'description'):
    print(f'{feature}: {getattr(cluster_value, feature)}')
print(cluster_value.meta.to_dict())

# Create \& Update Galaxies \& Clusters

PyMISPGalaxies provides a pythonic way of getting information about the existing Galaxies \& Clusters.  
However, since Galaxies \& Clusters are represented as immutable python dictionaries, we cannot modify them directly using the `Galaxies` \& `Clusters` classes.

We then need to add our changes to the JSON definition files.  
We can nonetheless use the `Galaxy`, `Cluster`, `ClusterValue` \& `ClusterValueMeta` classes, which help defining data that is valid.

## Update an existing Cluster

### Create the new Cluster values

As an example, we want to update the `tea-matrix` cluster:

In [ ]:
from pymispgalaxies.api import ClusterValue
from uuid import uuid4

brewing_cluster_value = ClusterValue(
    {
        'value': 'Brewing time 5-6 min',
        'description': 'Brewing time 5-6 minutes',
        'meta': {
            'killchain': ['tea:black']
        },
        'uuid': uuid4().__str__()
    }
)

temperature_cluster_value = ClusterValue(
    {
        'value': 'Water temp 100 degC',
        'description': 'Water temperature 100 degC',
        'meta': {
            'killchain': ['tea:black']
        },
        'uuid': uuid4().__str__()
    }
)

### Update the Cluster with the new values

In [ ]:
cluster = clusters.get('tea-matrix')

print(f'The Cluster values before:\n{cluster.cluster_values}\n')

cluster.cluster_values[brewing_cluster_value.value] = brewing_cluster_value
cluster.cluster_values[temperature_cluster_value.value] = temperature_cluster_value

print(f'The updated Cluster values:\n{cluster.cluster_values}')

# we update the cluster version
cluster.version += 1

### Bump the updated Cluster

In [ ]:
import json
from pathlib import Path

# Optional step to store Cluster values by alphbetical order
cluster.cluster_values = {feature: cluster.cluster_values[feature] for feature in sorted(cluster.cluster_values)}

clusters_dir = Path(clusters.root_dir_clusters).resolve()
content_to_write = json.loads(cluster.to_json())
print(json.dumps(content_to_write, indent=4))

with open(clusters_dir / f'{cluster.type}.json', 'w') as f:
    f.write(
        json.dumps(
            content_to_write,
            indent=4
        )
    )

### We can now check the Galaxy Cluster is up-to-date

In [ ]:
clusters = Clusters()

print(list(clusters.get('tea-matrix').cluster_values.keys()))

## Create a new Galaxy

### We declare both the Galaxy and the Cluster

In [ ]:
from pymispgalaxies.api import Galaxy, Cluster

new_galaxy = Galaxy(
    {
        'type': 'firearm-models',
        'name': 'Firearm Models',
        'description': 'Models of firearm',
        'icon': 'map',
        'uuid': uuid4().__str__(),
        'version': 1
    }
)

new_cluster = Cluster(
    {
        'authors': ['John Doe'],
        'category': 'equipment',
        'description': 'Models of firearm',
        'name': 'Firearm Models',
        'source': 'Open sources',
        'type': 'firearm-models',
        'uuid': uuid4().__str__(),
        'version': 1,
        'values': [
            {
                'value': 'Glock 17',
                'description': 'Polymer-framed, short recoil-operated, locked-breech semi-automatic pistol',
                'meta': {
                    'killchain': ['handgun:pistol']
                },
                'uuid': uuid4().__str__()
            },
            {
                'value': 'AK-47',
                'description': 'Kalashnikov: gas-operated assault rifle',
                'meta': {
                    'killchain': ['long-gun:assault-rifle']
                },
                'uuid': uuid4().__str__()
            },
            {
                'value': 'M40A3',
                'description': 'The M40 rifle is a bolt-action sniper rifle used by the US Marines',
                'meta': {
                    'killchain': ['long-gun:sniper-rifle']
                },
                'uuid': uuid4().__str__()
            }
        ]
    }
)

### Bump the new content

In [ ]:
galaxies_dir = Path(galaxies.root_dir_galaxies).resolve()

with open(galaxies_dir / f'{new_galaxy.type}.json', 'w') as f:
    f.write(json.dumps(new_galaxy.to_dict(), indent=4))
    
with open(clusters_dir / f'{new_cluster.type}.json', 'w') as f:
    f.write(
        json.dumps(
            json.loads(new_cluster.to_json()),
            indent=4
        )
    )

### We can now check the created content is available

In [ ]:
galaxies = Galaxies()
print(list(galaxies.keys()))
print(f"\nFirearm Models Galaxy in the list of available galaxies: {'Firearm Models' in galaxies.keys()}\n")

clusters = Clusters()
print(list(clusters.keys()))
print(f"\nfirearm-models Cluster in the list of available clusters: {'firearm-models' in clusters.keys()}")